# Лабораторная работа №3

# Двойственный симплекс метод

### Выполнил: Яковлев Артур, гр. 853501

### Проверила: Костюкова О.И.

#### Вариант 28

Импортируем все необходимые модули

In [143]:
import numpy as np
import scipy.linalg as sla

Исходные данные для тестового примера:

In [98]:
test_dataset = {
    'A': np.array([[2., -3.,  4., 0., 2.,  0., -2., 7.],
                   [0.,  2., -2., 2., 1., -1.,  4., 3.],
                   [0.,  4.,  1., 1., 3.,  5.,  1., 2.]]),
    'b': np.array([0., 4., 18.]),
    'c': np.array([-2., 15., -11., 8., 3., -2., 13., 7.]),
    'y': np.array([-1., 4., 1.]),
    'jb': [0, 1, 2],
}

Реализуем итерацию нашего алгоритма. Данная функция принимает в себя все исходные данные, обратную матрицу $B$ и вектор $\delta$. Итерация состоит из 9 шагов, каждый из которых приведен в коде ниже.

In [144]:
def iteration(dataset, B, delta):  # B is a np.array, delta is Dict[int, float]
    #Step 1
    x_b = B.dot(dataset['b'])
    print(f'Current plan: {x_b}')
    #Step 2
    if np.all(x_b > -1e-9):
        return None, x_b
    #Step 3
    k = np.argmin(x_b)
    print(f'k: {k}')
    jn = [i for i in range(dataset['A'].shape[1]) if i not in dataset['jb']]
    print(f'jn: {jn}')
    mu = {j: B[k, :].dot(dataset['A'][:, j]) for j in jn}
    print(f'mu: {mu}')
    #Step 4
    sigma = {j: -delta[j] / mu[j] if mu[j] < 1e-9 else np.inf for j in jn}
    print(f'sigma: {sigma}')
    #Step 5, 6
    sigma0 = min(sigma, key=sigma.get)
    if sigma[sigma0] == np.inf:
        return None, None
    #Step 7
    idx = dataset['jb'][k]
    del dataset['jb'][k]
    dataset['jb'].append(sigma0)
    dataset['jb'].sort()
    print(f'New jb: {dataset["jb"]}')
    #Step 8
    delta.pop(sigma0)
    for key in delta:
        delta[key] += sigma[sigma0] * mu[key]
    delta[idx] = sigma[sigma0]
    print(f'New delta: {delta}')
    #Step 9
    B = sla.inv(dataset['A'][:, dataset['jb']])
    return B, delta

Далее реализуем сам алгоритм решения. Функция принимает в себя набор исходных данных, состоящих из матрицы $A$, векторов $b$, $c$, $y$ и массива индексов $J_b$, а также максимальное число итераций алгоритма (по умолчанию 20). В основном цикле программы с помощью функции итерации обновляются матрица $B$ и вектор $\delta$ и осуществляется вывод оптимального решения в случае, если оно было получено.

In [156]:
def solve(dataset, max_iter=20):
    B = sla.inv(dataset['A'][:, dataset['jb']])
    jn = [i for i in range(dataset['A'].shape[1]) if i not in dataset['jb']]
    init_delta = np.dot(dataset['c'][dataset['jb']], B @ dataset['A'][:, jn]) - dataset['c'][jn]
    delta = {jn[i]: init_delta[i] for i in range(len(jn))}
    for i in range(max_iter):
        print('\n==============================================\n')
        print(f'Iteration {i + 1}\n')
        B, delta = iteration(dataset, B, delta)
        if B is None:
            if delta is None:
                print('No solution')
            else:
                plan = np.zeros(dataset['c'].shape)
                plan[dataset['jb']] = delta
                print(f'\nOptimized plan: {plan}')
                print(f"cx: {dataset['c'][dataset['jb']].dot(delta)}")
            return
    print(f'\nBest found plan: {delta}')

Проверим наш алгоритм на тестовом наборе данных

In [157]:
solve(test_dataset)



Iteration 1

Current plan: [2. 4. 2.]

Optimized plan: [2. 4. 2. 0. 0. 0. 0. 0.]
cx: 34.0


Далее рассмотрим задачу для варианта 28. Исходные данные:

In [158]:
var28 = {
    'A': np.array([[ 1.,  2., -1.,  0.,  3.,  2.,  3.,  1.,  0.],
                   [ 1.,  0.,  0.,  1., -1.,  3.,  0.,  7.,  5.],
                   [ 0.,  1., -4.,  1., -3.,  1.,  0.,  1.,  0.],
                   [ 1.,  2., -3.,  1., -1.,  2.,  5.,  0.,  4.]]),
    'b': np.array([28., 59., -3., 3.]),
    'c': np.array([8., 8., 2., 0., 21., 12., 16., 16., 9.]),
    'y': np.array([5., 2., -3., 1.]),
    'jb': [1, 3, 4, 6],  # изменил несколько индексов, чтобы алгоритм не прекращал работу на первой итерации
}

In [159]:
solve(var28)



Iteration 1

Current plan: [-13.65217391  83.17391304  24.17391304  -5.73913043]
k: 0
jn: [0, 2, 5, 7, 8]
mu: {0: -0.21739130434782605, 2: -2.2608695652173916, 5: -0.17391304347826075, 7: -1.652173913043478, 8: -3.0}
sigma: {0: -0.6000000000000069, 2: 1.346153846153845, 5: 21.24999999999999, 7: 2.7894736842105314, 8: 1.3333333333333333}
New jb: [0, 3, 4, 6]
New delta: {2: 4.400000000000013, 5: 3.799999999999997, 7: 5.600000000000019, 8: 5.80000000000002, 1: -0.6000000000000069}


Iteration 2

Current plan: [ 62.8  -4.2  -0.4 -11.2]
k: 3
jn: [1, 2, 5, 7, 8]
mu: {1: 0.4, 2: -0.6000000000000001, 5: -0.2, 7: -1.4000000000000001, 8: -0.2}
sigma: {1: inf, 2: 7.3333333333333535, 5: 18.999999999999986, 7: 4.000000000000013, 8: 29.0000000000001}
New jb: [0, 3, 4, 7]
New delta: {2: 2.0000000000000044, 5: 2.9999999999999947, 8: 5.000000000000018, 1: 0.9999999999999986, 6: 4.000000000000013}


Iteration 3

Current plan: [2. 7. 6. 8.]

Optimized plan: [2. 0. 0. 7. 6. 0. 0. 8. 0.]
cx: 270.0


Аналогично решим задачу для варианта 29.

In [162]:
var29 = {
    'A': np.array([
         [-2, -1,  1,  -7,  1,  0,  0,  2],
         [-4,  2,  1,   0,  5,  1, -1,  5],
         [ 1,  1,  0,  -1,  0,  3, -1,  1]]),
    'b': np.array([-2, 4, -2]),
    'c': np.array([-12, 2, 2, -6, 10, -1, -9, 8]),
    'y': np.array([1, 2, -1]),
    'jb': [1, 3, 5]
}

In [163]:
solve(var29)



Iteration 1

Current plan: [ 2.82352941 -0.11764706 -1.64705882]
k: 2
jn: [0, 2, 4, 6, 7]
mu: {0: 1.4705882352941178, 2: -0.29411764705882354, 4: -1.2352941176470589, 6: -0.1764705882352941, 7: -0.8823529411764707}
sigma: {0: inf, 2: 3.4, 4: 0.8095238095238095, 6: 45.333333333333336, 7: 3.3999999999999995}
New jb: [1, 3, 4]
New delta: {0: 2.1904761904761907, 2: 0.7619047619047619, 6: 7.857142857142857, 7: 2.2857142857142856, 5: 0.8095238095238095}


Iteration 2

Current plan: [-1.33333333  0.66666667  1.33333333]
k: 0
jn: [0, 2, 5, 6, 7]
mu: {0: 0.9761904761904759, 2: -0.09523809523809519, 5: 2.5238095238095237, 6: -0.8571428571428571, 7: 0.7142857142857144}
sigma: {0: inf, 2: 8.000000000000004, 5: inf, 6: 9.166666666666666, 7: inf}
New jb: [2, 3, 4]
New delta: {0: 10.000000000000002, 6: 0.9999999999999973, 7: 8.000000000000004, 5: 21.00000000000001, 1: 8.000000000000004}


Iteration 3

Current plan: [14.  2. -2.]
k: 2
jn: [0, 1, 5, 6, 7]
mu: {0: 1.25, 1: 2.5, 5: 5.5, 6: -2.0, 7: 2.5